### rocket science

### to rewrite the cell below, bugs

In [2]:
# --- Pawn detection w/ estimated BG (memory-safe) + choose frame index ---

import cv2
import numpy as np
import heapq
import matplotlib.pyplot as plt

from detect_board import (
    CalibCFG, estimate_homography, estimate_inner_box_median, build_masks,
    warp_board, diff_warped_vs_bg_clean
)
from detect_pawns import PawnCFG, detect_pawns_from_diff

# ----------------------------
# 0) Inputs you control
# ----------------------------
video_path = "data/easy/3_easy.mp4"   # <-- change
warp_size  = 900                      # <-- your project warp size

frame_idx_to_detect = 0       # <-- YOU set this (0-based)
bg_start_frame = 0                    # <-- optional: where BG sampling starts
# ----------------------------

# ----------------------------
# 1) Calibrate board & masks
# ----------------------------
cfg = CalibCFG(video_path=video_path, warp_size=warp_size, show_debug=False, save_debug=False)
H, _ = estimate_homography(video_path, cfg)
inner_box = estimate_inner_box_median(video_path, H, cfg)
outer_mask, inner_mask, ring_mask, dice_roi, dice_roi_mask = build_masks(warp_size, inner_box)

# ----------------------------
# 2) Memory-safe background estimation (keeps only K cleanest frames)
# ----------------------------
def compute_clean_background_light(
    video_path: str,
    H: np.ndarray,
    warp_size: int,
    *,
    roi_mask: np.ndarray | None = None,
    start_frame: int = 0,
    n_samples: int = 180,
    stride: int = 5,
    keep_k: int = 30,
    mog2_history: int = 300,
    mog2_varThreshold: int = 14,
    open_k: int = 3,
    close_k: int = 9,
    min_visible_count: int = 4,
):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")

    # Seek to deterministic starting point
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(start_frame))

    mog = cv2.createBackgroundSubtractorMOG2(
        history=mog2_history,
        varThreshold=mog2_varThreshold,
        detectShadows=False
    )

    heap = []  # (-ratio, warped_bgr, fg_u8)

    roi_denom = float(np.count_nonzero(roi_mask)) if roi_mask is not None else None
    if roi_mask is not None and roi_denom == 0:
        raise ValueError("roi_mask has zero area")

    i = 0
    collected = 0
    while collected < n_samples:
        ok, frame = cap.read()
        if not ok:
            break

        if i % stride == 0:
            warped = warp_board(frame, H, warp_size)

            fg = mog.apply(warped)
            fg = (fg > 0).astype(np.uint8) * 255

            if open_k > 1:
                k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_k, open_k))
                fg = cv2.morphologyEx(fg, cv2.MORPH_OPEN, k, iterations=1)
            if close_k > 1:
                k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_k, close_k))
                fg = cv2.morphologyEx(fg, cv2.MORPH_CLOSE, k, iterations=1)

            if roi_mask is not None:
                fg_roi = cv2.bitwise_and(fg, fg, mask=roi_mask)
                ratio = float(np.count_nonzero(fg_roi)) / roi_denom
            else:
                ratio = float(np.count_nonzero(fg)) / float(fg.size)

            item = (-ratio, warped, fg)

            if len(heap) < keep_k:
                heapq.heappush(heap, item)
            else:
                # replace worst if current is cleaner
                if item[0] > heap[0][0]:
                    heapq.heapreplace(heap, item)

            collected += 1

        i += 1

    cap.release()

    if len(heap) < max(10, keep_k // 2):
        raise RuntimeError(f"Too few frames kept for BG (kept={len(heap)}). Lower keep_k/stride or increase n_samples.")

    kept = sorted([(-nr, fr, fg) for (nr, fr, fg) in heap], key=lambda x: x[0])
    frames_k = np.stack([x[1] for x in kept], axis=0).astype(np.uint8)  # (k,H,W,3)
    fgs_k    = np.stack([x[2] for x in kept], axis=0).astype(np.uint8)  # (k,H,W)

    mask_bg = (fgs_k == 0)
    visible_count = mask_bg.sum(axis=0)

    bg = np.zeros_like(frames_k[0], dtype=np.uint8)
    for c in range(3):
        vals = frames_k[..., c].astype(np.float32)
        vals[~mask_bg] = np.nan
        med = np.nanmedian(vals, axis=0)
        bg[..., c] = np.nan_to_num(med, nan=0).astype(np.uint8)

    holes = (visible_count < int(min_visible_count)).astype(np.uint8) * 255
    if np.count_nonzero(holes) > 0:
        bg = cv2.inpaint(bg, holes, 3, cv2.INPAINT_TELEA)

    return bg

bg_clean = compute_clean_background_light(
    video_path, H, warp_size,
    roi_mask=ring_mask,         # focus on pawn ring stability
    start_frame=bg_start_frame, # YOU control this too
    n_samples=180,
    stride=3,
    keep_k=30,
)

# ----------------------------
# 3) Read the exact frame you want
# ----------------------------
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_idx_to_detect))
ok, frame_bgr = cap.read()
cap.release()
if not ok:
    raise RuntimeError(f"Could not read frame index {frame_idx_to_detect} from {video_path}")

# ----------------------------
# 4) Diff + pawn detection
# ----------------------------
diff_bgr, _ = diff_warped_vs_bg_clean(
    frame_bgr, H, warp_size, bg_clean,
    to_gray=False,
    return_stats=False
)

pawns, fg_mask, dbg = detect_pawns_from_diff(
    diff_bgr,
    ring_mask=ring_mask,
    prev_centers=None,
    cfg=PawnCFG(),
)

print("Frame:", frame_idx_to_detect)
print("Pawns:", pawns)
print("Thr:", dbg["thr"], "Contours:", dbg["n_contours"], "Candidates:", dbg["n_candidates"])

# ----------------------------
# 5) Viz (single figure)
# ----------------------------
warped = warp_board(frame_bgr, H, warp_size)

viz = warped.copy()
for i, p in enumerate(pawns):
    x1, y1, x2, y2 = p["bbox"]
    cv2.rectangle(viz, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cx, cy = p["center"]
    cv2.circle(viz, (int(cx), int(cy)), 6, (0, 255, 0), -1)
    cv2.putText(viz, f"pawn{i+1}", (x1, max(12, y1 - 6)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

plt.figure(figsize=(14, 6))
plt.suptitle(f"Pawn detection @ frame {frame_idx_to_detect}")
plt.subplot(1,2,1); plt.title("Warped + overlay"); plt.imshow(cv2.cvtColor(viz, cv2.COLOR_BGR2RGB)); plt.axis("off")
plt.subplot(1,2,2); plt.title("FG mask (ring)"); plt.imshow(fg_mask, cmap="gray"); plt.axis("off")
plt.tight_layout()
plt.show()


RuntimeError: Board quad not detected reliably. Tune canny/area/approx parameters or improve lighting.

In [4]:
# --- Interactive: pick video + frame, warp board using your calibration, detect cards only in inner field ---

import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt

import importlib.util
from dataclasses import asdict

import ipywidgets as widgets
from IPython.display import display, clear_output

# -------- load your script as a module --------
SCRIPT_PATH = "detect_board.py"  # provided path
spec = importlib.util.spec_from_file_location("detect_board", SCRIPT_PATH)
detect_board = importlib.util.module_from_spec(spec)
spec.loader.exec_module(detect_board)



CalibCFG = detect_board.CalibCFG

# -------- utilities --------
def bgr_to_rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def show_bgr(img, title=None, figsize=(10,6)):
    plt.figure(figsize=figsize)
    if title: plt.title(title)
    plt.imshow(bgr_to_rgb(img))
    plt.axis("off")
    plt.show()

def show_gray(img, title=None, figsize=(10,4)):
    plt.figure(figsize=figsize)
    if title: plt.title(title)
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    plt.show()

def list_videos(root="data/easy"):
    exts = ("*.mp4", "*.mov", "*.mkv", "*.avi", "*.m4v")
    out = []
    for ext in exts:
        out += glob.glob(os.path.join(root, "**", ext), recursive=True)
    out = sorted(out)
    return out

def get_video_meta(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        cap.release()
        raise RuntimeError(f"Cannot open video: {video_path}")
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
    fps = float(cap.get(cv2.CAP_PROP_FPS) or 0.0)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)
    cap.release()
    return nframes, fps, w, h

def read_frame_at(video_path, idx):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        cap.release()
        raise RuntimeError(f"Cannot open video: {video_path}")
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
    ok, frame = cap.read()
    cap.release()
    if not ok or frame is None:
        raise RuntimeError(f"Cannot read frame {idx} from {video_path}")
    return frame

# -------- card detection limited to inner crop --------
def detect_cards_in_crop(
    crop_bgr,
    *,
    blur_ksize=5,
    canny1=60,
    canny2=160,
    dilate_iter=1,
    erode_iter=0,
    min_area=2000,
    max_area=200000,
    min_aspect=0.55,
    max_aspect=1.8,
    approx_eps_frac=0.02,
    use_hsv_filter=True,
    h_min=0, h_max=179,
    s_min=0, s_max=255,
    v_min=0, v_max=255,
    hsv_keep_ratio_min=0.15,
):
    img = crop_bgr.copy()
    k = int(blur_ksize)
    if k % 2 == 0: k += 1
    k = max(3, k)
    blurred = cv2.GaussianBlur(img, (k, k), 0)

    edges = cv2.Canny(blurred, int(canny1), int(canny2))
    morph = edges.copy()
    if dilate_iter > 0:
        morph = cv2.dilate(morph, None, iterations=int(dilate_iter))
    if erode_iter > 0:
        morph = cv2.erode(morph, None, iterations=int(erode_iter))

    cnts, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_mask = None
    if use_hsv_filter:
        lower = np.array([h_min, s_min, v_min], dtype=np.uint8)
        upper = np.array([h_max, s_max, v_max], dtype=np.uint8)
        hsv_mask = cv2.inRange(hsv, lower, upper)

    boxes, quads = [], []
    for c in cnts:
        area = cv2.contourArea(c)
        if area < min_area or area > max_area:
            continue

        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, float(approx_eps_frac) * peri, True)
        if len(approx) != 4 or not cv2.isContourConvex(approx):
            continue

        x, y, w, h = cv2.boundingRect(approx)
        if w <= 0 or h <= 0:
            continue
        asp = w / float(h)
        if not (min_aspect <= asp <= max_aspect):
            continue

        if use_hsv_filter and hsv_mask is not None:
            quad_mask = np.zeros(img.shape[:2], dtype=np.uint8)
            cv2.fillConvexPoly(quad_mask, approx.reshape(-1, 2), 255)
            inside = cv2.bitwise_and(hsv_mask, hsv_mask, mask=quad_mask)
            keep_ratio = (inside > 0).sum() / max(1, (quad_mask > 0).sum())
            if keep_ratio < hsv_keep_ratio_min:
                continue

        boxes.append((x, y, w, h))
        quads.append(approx)

    overlay = img.copy()
    cv2.drawContours(overlay, quads, -1, (0,255,0), 2)
    for (x,y,w,h) in boxes:
        cv2.rectangle(overlay, (x,y), (x+w, y+h), (255,0,0), 2)

    dbg = {"edges": edges, "morph": morph, "hsv_mask": hsv_mask, "overlay": overlay}
    return boxes, quads, dbg


# -------- calibration cache per video --------
_calib_cache = {}

def get_or_build_calibration(video_path, warp_size=900, show_debug=False):
    key = (video_path, warp_size)
    if key in _calib_cache:
        return _calib_cache[key]

    cfg = CalibCFG(video_path=video_path, warp_size=warp_size, show_debug=show_debug, save_debug=False)

    # 1) estimate board homography
    H, quad_med = detect_board.estimate_homography(video_path, cfg)

    # 2) estimate inner box
    inner_box = detect_board.estimate_inner_box_median(video_path, H, cfg)

    # 3) build masks (we mainly need inner_box)
    outer_mask, inner_mask, ring_mask, dice_roi, dice_roi_mask = detect_board.build_masks(warp_size, inner_box)

    calib = {
        "cfg": cfg,
        "H": H,
        "quad_med": quad_med,
        "inner_box": inner_box,
        "inner_mask": inner_mask,
        "ring_mask": ring_mask
    }
    _calib_cache[key] = calib
    return calib


# -------- UI --------
videos = list_videos("data")
if not videos:
    raise RuntimeError("No videos found under data/. Put your mp4 in data/...")

w_video = widgets.Dropdown(
    options=videos,
    value=videos[0],
    description="video",
    layout=widgets.Layout(width="80%")
)

w_warp = widgets.IntSlider(value=900, min=600, max=1400, step=50, description="warp")

# detection knobs
w_blur = widgets.IntSlider(value=5, min=3, max=21, step=2, description="blur")
w_c1 = widgets.IntSlider(value=60, min=0, max=300, step=1, description="canny1")
w_c2 = widgets.IntSlider(value=160, min=0, max=400, step=1, description="canny2")
w_dil = widgets.IntSlider(value=1, min=0, max=10, step=1, description="dilate")
w_ero = widgets.IntSlider(value=0, min=0, max=10, step=1, description="erode")
w_eps = widgets.FloatSlider(value=0.02, min=0.001, max=0.08, step=0.001, description="eps")

w_minA = widgets.IntSlider(value=2000, min=0, max=400000, step=200, description="min_area")
w_maxA = widgets.IntSlider(value=200000, min=1000, max=2500000, step=2000, description="max_area")
w_minAsp = widgets.FloatSlider(value=0.55, min=0.1, max=3.0, step=0.01, description="min_aspect")
w_maxAsp = widgets.FloatSlider(value=1.8, min=0.1, max=5.0, step=0.01, description="max_aspect")

w_useHSV = widgets.Checkbox(value=True, description="HSV filter")
w_hmin = widgets.IntSlider(value=0, min=0, max=179, step=1, description="H min")
w_hmax = widgets.IntSlider(value=179, min=0, max=179, step=1, description="H max")
w_smin = widgets.IntSlider(value=0, min=0, max=255, step=1, description="S min")
w_smax = widgets.IntSlider(value=255, min=0, max=255, step=1, description="S max")
w_vmin = widgets.IntSlider(value=0, min=0, max=255, step=1, description="V min")
w_vmax = widgets.IntSlider(value=255, min=0, max=255, step=1, description="V max")
w_keep = widgets.FloatSlider(value=0.15, min=0.0, max=1.0, step=0.01, description="HSV keep%")

w_view = widgets.Dropdown(
    options=["warped+inner", "crop overlay", "edges", "morph", "hsv_mask"],
    value="crop overlay",
    description="view"
)

w_frame = widgets.IntSlider(value=0, min=0, max=1, step=1, description="frame")

out = widgets.Output()

def _refresh_frame_slider(*_):
    try:
        nframes, fps, vw, vh = get_video_meta(w_video.value)
    except Exception as e:
        with out:
            clear_output(wait=True)
            print(e)
        return

    # set a sane max
    if nframes <= 0:
        nframes = 3000
    w_frame.max = max(0, nframes - 1)
    w_frame.value = min(w_frame.value, w_frame.max)

_refresh_frame_slider()

def _run(*_):
    with out:
        clear_output(wait=True)

        video_path = w_video.value
        warp_size = int(w_warp.value)

        # build or reuse calibration for this video
        try:
            calib = get_or_build_calibration(video_path, warp_size=warp_size, show_debug=False)
        except Exception as e:
            print("Calibration failed:", e)
            print("Tip: tune CalibCFG knobs in your script if needed (canny/area/approx).")
            return

        H = calib["H"]
        inner_box = calib["inner_box"]
        xl, yt, xr, yb = inner_box

        # read chosen frame
        try:
            frame = read_frame_at(video_path, int(w_frame.value))
        except Exception as e:
            print(e)
            return

        warped = detect_board.warp_board(frame, H, warp_size)
        crop = warped[yt:yb, xl:xr].copy()

        # detect only inside inner crop
        boxes, quads, dbg = detect_cards_in_crop(
            crop,
            blur_ksize=w_blur.value,
            canny1=w_c1.value,
            canny2=w_c2.value,
            dilate_iter=w_dil.value,
            erode_iter=w_ero.value,
            approx_eps_frac=w_eps.value,
            min_area=w_minA.value,
            max_area=w_maxA.value,
            min_aspect=w_minAsp.value,
            max_aspect=w_maxAsp.value,
            use_hsv_filter=w_useHSV.value,
            h_min=w_hmin.value, h_max=w_hmax.value,
            s_min=w_smin.value, s_max=w_smax.value,
            v_min=w_vmin.value, v_max=w_vmax.value,
            hsv_keep_ratio_min=w_keep.value,
        )

        # draw detections back onto warped
        warped_overlay = warped.copy()
        # inner box
        cv2.rectangle(warped_overlay, (xl, yt), (xr, yb), (0, 255, 255), 2)

        # offset crop detections into warped coords
        for q in quads:
            q2 = q.copy()
            q2[:, 0, 0] += xl
            q2[:, 0, 1] += yt
            cv2.drawContours(warped_overlay, [q2], -1, (0, 255, 0), 2)

        for (x,y,w,h) in boxes:
            cv2.rectangle(warped_overlay, (xl+x, yt+y), (xl+x+w, yt+y+h), (255, 0, 0), 2)

        print(f"Video: {video_path}")
        print(f"Frame: {w_frame.value}")
        print(f"Inner box (xl,yt,xr,yb): {inner_box}")
        print(f"Detections in inner field: {len(boxes)}")

        view = w_view.value
        if view == "warped+inner":
            show_bgr(warped_overlay, "Warped board + inner box + detections", figsize=(9,9))
        elif view == "crop overlay":
            show_bgr(dbg["overlay"], "Inner crop detections overlay", figsize=(9,6))
        elif view == "edges":
            show_gray(dbg["edges"], "Inner crop Canny edges")
        elif view == "morph":
            show_gray(dbg["morph"], "Inner crop edges after morphology")
        else:
            if dbg["hsv_mask"] is None:
                print("HSV mask disabled.")
            else:
                show_gray(dbg["hsv_mask"], "Inner crop HSV inRange mask")

ui = widgets.VBox([
    widgets.HBox([w_video, w_warp]),
    w_frame,
    widgets.HBox([w_view, w_useHSV]),
    widgets.HBox([w_blur, w_c1, w_c2]),
    widgets.HBox([w_dil, w_ero, w_eps]),
    widgets.HBox([w_minA, w_maxA]),
    widgets.HBox([w_minAsp, w_maxAsp]),
    widgets.HBox([w_hmin, w_hmax]),
    widgets.HBox([w_smin, w_smax]),
    widgets.HBox([w_vmin, w_vmax]),
    w_keep,
])

def _on_video_change(*_):
    _refresh_frame_slider()
    _run()

w_video.observe(_on_video_change, names="value")
w_warp.observe(_on_video_change, names="value")

for w in [w_frame, w_view, w_useHSV, w_blur, w_c1, w_c2, w_dil, w_ero, w_eps,
          w_minA, w_maxA, w_minAsp, w_maxAsp,
          w_hmin, w_hmax, w_smin, w_smax, w_vmin, w_vmax, w_keep]:
    w.observe(_run, names="value")

display(ui, out)
_run()


Output()

### cards detection correct.

In [3]:
# --- Run card detection (STRICTLY detect_cards.py logic) on a video and save annotated warped video ---

import os
import cv2

from detect_board import CalibCFG, estimate_homography, estimate_inner_box_median, warp_board
from detect_cards import CardDetectCFG, detect_cards_in_inner_field

# ============
# CONFIG
# ============
VIDEO_PATH = "data/easy/2_easy.mp4"   # <- set this
WARP_SIZE  = 900                     # keep same as your board warp convention
OUT_DIR    = "out_cards"
os.makedirs(OUT_DIR, exist_ok=True)

out_path = os.path.join(OUT_DIR, os.path.splitext(os.path.basename(VIDEO_PATH))[0] + "_cards_annotated_warped.mp4")

# ============
# CALIBRATION (frame -> warped) + inner box
# ============
cfg_calib = CalibCFG(video_path=VIDEO_PATH, warp_size=WARP_SIZE, show_debug=False, save_debug=False)
H, _ = estimate_homography(VIDEO_PATH, cfg_calib)
inner_box = estimate_inner_box_median(VIDEO_PATH, H, cfg_calib)

# ============
# CARD DETECTION CONFIG (defaults from detect_cards.py)
# ============
cfg_cards = CardDetectCFG()  # uses the file's default parameters

# ============
# PROCESS VIDEO
# ============
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps <= 0:
    fps = 30.0

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
vw = cv2.VideoWriter(out_path, fourcc, fps, (WARP_SIZE, WARP_SIZE))

while True:
    ok, frame = cap.read()
    if not ok:
        break

    warped = warp_board(frame, H, WARP_SIZE)

    # STRICT: detect directly on warped crop, no background subtraction
    det = detect_cards_in_inner_field(warped, inner_box, cfg=cfg_cards)

    # Draw boxes/quads exactly using detect_cards.py helpers (+ inner box)
    overlay = det.draw_on_warped(warped, inner_box=inner_box, draw_inner_box=True)

    vw.write(overlay)

cap.release()
vw.release()

print("Saved:", out_path)
print("inner_box:", inner_box)


KeyboardInterrupt: 

In [ ]:
# --- Interactive debug: tune dice + pips parameters on a single frame (IMPROVED) ---

import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

from detect_board import CalibCFG, estimate_homography, estimate_inner_box_median, warp_board

from detect_dice import (
    DiceRegionCFG,
    TopFaceCropCFG,
    PipCountCFG,
    detect_dice_in_inner_field,
    read_top_face_pips,
    draw_dice_on_warped,
)

# ======================
# 0) Set your video here
# ======================
VIDEO_PATH = "data/easy/2_easy.mp4"  # <- change this
WARP_SIZE = 900

# ======================
# 1) One-time calibration
# ======================
cfg_calib = CalibCFG(video_path=VIDEO_PATH, warp_size=WARP_SIZE, show_debug=False, save_debug=False)
H, _ = estimate_homography(VIDEO_PATH, cfg_calib)
inner_box = estimate_inner_box_median(VIDEO_PATH, H, cfg_calib)

def get_warped_frame(frame_idx: int):
    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {VIDEO_PATH}")
    cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_idx))
    ok, frame = cap.read()
    cap.release()
    if not ok:
        raise RuntimeError(f"Could not read frame {frame_idx}")
    return warp_board(frame, H, WARP_SIZE)

# ======================
# 2) Widgets
# ======================
frame_idx_w = widgets.IntSlider(value=0, min=0, max=3000, step=1, description="frame", continuous_update=False)

# Dice region (inner field) detection params (better defaults for tiny dice)
ab_thr_w   = widgets.IntSlider(value=38,  min=10, max=80, step=1, description="region ab_thr", continuous_update=False)
L_min_w    = widgets.IntSlider(value=170, min=50, max=245, step=1, description="region L_min", continuous_update=False)
L_q_w      = widgets.IntSlider(value=92,  min=50, max=99, step=1, description="region L_q", continuous_update=False)
open_k_w   = widgets.IntSlider(value=5,   min=1, max=31, step=2, description="region open_k", continuous_update=False)
close_k_w  = widgets.IntSlider(value=13,  min=1, max=51, step=2, description="region close_k", continuous_update=False)
close_it_w = widgets.IntSlider(value=1,   min=1, max=6,  step=1, description="region close_it", continuous_update=False)

min_area_w  = widgets.FloatSlider(value=0.00025, min=0.00005, max=0.01, step=0.00005,
                                  description="min_area_frac", readout_format=".5f", continuous_update=False)
max_area_w  = widgets.FloatSlider(value=0.06, min=0.01, max=0.30, step=0.01,
                                  description="max_area_frac", readout_format=".2f", continuous_update=False)
max_aspect_w = widgets.FloatSlider(value=1.35, min=1.05, max=3.00, step=0.05,
                                   description="max_aspect", readout_format=".2f", continuous_update=False)
pad_frac_w = widgets.FloatSlider(value=0.06, min=0.00, max=0.20, step=0.01,
                                 description="pad_frac", readout_format=".2f", continuous_update=False)

# New: rotation-invariant square checks
use_rot_w = widgets.Checkbox(value=True, description="use_rotated_rect")
max_rot_aspect_w = widgets.FloatSlider(value=1.25, min=1.00, max=2.00, step=0.05,
                                       description="max_rot_aspect", readout_format=".2f", continuous_update=False)
min_extent_w = widgets.FloatSlider(value=0.55, min=0.10, max=0.95, step=0.05,
                                   description="min_extent", readout_format=".2f", continuous_update=False)
require_quad_w = widgets.Checkbox(value=False, description="require_quad_like")

# Selection / scoring
keep_top_k_w = widgets.IntSlider(value=2, min=1, max=4, step=1, description="keep_top_k", continuous_update=False)
max_score_cand_w = widgets.IntSlider(value=12, min=1, max=30, step=1, description="max_to_score", continuous_update=False)
score_with_pips_w = widgets.Checkbox(value=True, description="score_with_pips")

# Top-face crop params (used before pip counting)
tf_ab_thr_w   = widgets.IntSlider(value=35,  min=10, max=80, step=1, description="face ab_thr", continuous_update=False)
tf_L_min_w    = widgets.IntSlider(value=175, min=50, max=245, step=1, description="face L_min", continuous_update=False)
tf_L_q_w      = widgets.IntSlider(value=94,  min=50, max=99, step=1, description="face L_q", continuous_update=False)
tf_open_w     = widgets.IntSlider(value=7,   min=1, max=31, step=2, description="face open_k", continuous_update=False)
tf_close_w    = widgets.IntSlider(value=15,  min=1, max=51, step=2, description="face close_k", continuous_update=False)
tf_close_it_w = widgets.IntSlider(value=1,   min=1, max=6,  step=1, description="face close_it", continuous_update=False)
tf_pad_w      = widgets.FloatSlider(value=0.03, min=0.00, max=0.15, step=0.01,
                                    description="face pad", readout_format=".2f", continuous_update=False)
tf_out_w      = widgets.IntSlider(value=650, min=200, max=900, step=10, description="face out_size", continuous_update=False)

# Pip counting params
pip_ab_thr_w  = widgets.IntSlider(value=55,  min=10, max=100, step=1, description="pip ab_thr", continuous_update=False)
pip_L_lo_w    = widgets.IntSlider(value=165, min=50, max=245, step=1, description="pip L_lo", continuous_update=False)
pip_open_w    = widgets.IntSlider(value=3,   min=1, max=21, step=2, description="pip open_k", continuous_update=False)
pip_close_w   = widgets.IntSlider(value=9,   min=1, max=41, step=2, description="pip close_k", continuous_update=False)
pip_close_it_w = widgets.IntSlider(value=1,  min=1, max=6,  step=1, description="pip close_it", continuous_update=False)
border_w      = widgets.FloatSlider(value=0.045, min=0.0, max=0.12, step=0.005,
                                    description="border_frac", readout_format=".3f", continuous_update=False)
min_pip_a_w   = widgets.FloatSlider(value=0.0012, min=0.0002, max=0.01, step=0.0002,
                                    description="min_pip_area", readout_format=".4f", continuous_update=False)
max_pip_a_w   = widgets.FloatSlider(value=0.06, min=0.01, max=0.20, step=0.01,
                                    description="max_pip_area", readout_format=".2f", continuous_update=False)

# Display toggles
show_masks_w = widgets.Checkbox(value=True, description="show masks")
show_faces_w = widgets.Checkbox(value=True, description="show top faces & pips")
show_roi_color_w = widgets.Checkbox(value=True, description="show inner ROI (color)")
show_scored_w = widgets.Checkbox(value=True, description="print scored candidates")

# ======================
# 3) Debug runner
# ======================
_cached = {"frame_idx": None, "warped": None}

def run_debug(
    frame,
    region_ab_thr, region_L_min, region_L_q, region_open_k, region_close_k, region_close_it,
    min_area_frac, max_area_frac, max_aspect, pad_frac,
    use_rotated_rect, max_rot_aspect, min_extent, require_quad_like,
    keep_top_k, max_to_score, score_with_pips,
    face_ab_thr, face_L_min, face_L_q, face_open_k, face_close_k, face_close_it, face_pad, face_out_size,
    pip_ab_thr, pip_L_lo, pip_open_k, pip_close_k, pip_close_it, border_frac, min_pip_area, max_pip_area,
    show_masks, show_faces, show_roi_color, show_scored
):
    # Load & warp frame only when frame_idx changes
    if _cached["frame_idx"] != frame:
        _cached["warped"] = get_warped_frame(frame)
        _cached["frame_idx"] = frame
    warped = _cached["warped"]

    region_cfg = DiceRegionCFG(
        ab_thr=int(region_ab_thr),
        L_min=int(region_L_min),
        L_quantile=int(region_L_q),
        open_k=int(region_open_k),
        close_k=int(region_close_k),
        close_iter=int(region_close_it),
        min_area_frac=float(min_area_frac),
        max_area_frac=float(max_area_frac),
        max_aspect=float(max_aspect),
        pad_frac=float(pad_frac),
        keep_top_k=int(keep_top_k),

        use_rotated_rect=bool(use_rotated_rect),
        max_rot_aspect=float(max_rot_aspect),
        min_extent=float(min_extent),
        require_quad_like=bool(require_quad_like),

        run_pips=True,                 # ✅ needed for scoring
        score_with_pips=bool(score_with_pips),
        max_candidates_to_score=int(max_to_score),

        debug_keep_color_roi=True,
    )

    face_cfg = TopFaceCropCFG(
        out_size=int(face_out_size),
        upscale_long_side=int(face_out_size),
        ab_thr=int(face_ab_thr),
        L_min=int(face_L_min),
        L_quantile=int(face_L_q),
        open_k=int(face_open_k),
        close_k=int(face_close_k),
        close_iter=int(face_close_it),
        pad_frac=float(face_pad),
        debug=False,
    )

    pip_cfg = PipCountCFG(
        ab_thr=int(pip_ab_thr),
        L_lo=int(pip_L_lo),
        open_k=int(pip_open_k),
        close_k=int(pip_close_k),
        close_iter=int(pip_close_it),
        border_frac=float(border_frac),
        min_area_frac=float(min_pip_area),
        max_area_frac=float(max_pip_area),
        debug=False,
    )

    det = detect_dice_in_inner_field(warped, inner_box, region_cfg=region_cfg, crop_cfg=face_cfg, pip_cfg=pip_cfg)
    annot = draw_dice_on_warped(warped, det, inner_box=inner_box)

    # Print quick summary
    print("frame:", frame)
    print("inner_box:", inner_box)
    print("dice boxes (warped):", det.boxes_warped)

    # Print candidate scoring info from region stage
    if show_scored and det.debug is not None and "scored_candidates" in det.debug:
        print("\nTop scored candidates (pre-final selection):")
        for d in det.debug["scored_candidates"][: min(10, len(det.debug["scored_candidates"]))]:
            print(d)

    # Compute pips for each detected crop with full debug (face + masks)
    dice_vals = []
    full = []
    if det.crops is not None:
        for crop in det.crops:
            r = read_top_face_pips(crop, crop_cfg=face_cfg, pip_cfg=pip_cfg)
            dice_vals.append(int(r["pip_count"]))
            full.append(r)
    print("dice values:", dice_vals)

    # --- Display: annotated + masks + inner ROI color ---
    if show_masks and det.debug is not None:
        fig = plt.figure(figsize=(18, 6))
        ax1 = fig.add_subplot(1, 4, 1)
        ax1.set_title("annotated warped")
        ax1.imshow(cv2.cvtColor(annot, cv2.COLOR_BGR2RGB))
        ax1.axis("off")

        ax2 = fig.add_subplot(1, 4, 2)
        ax2.set_title("region mask (raw)")
        ax2.imshow(det.debug["mask_raw"], cmap="gray")
        ax2.axis("off")

        ax3 = fig.add_subplot(1, 4, 3)
        ax3.set_title("region mask (morph)")
        ax3.imshow(det.debug["mask_morph"], cmap="gray")
        ax3.axis("off")

        ax4 = fig.add_subplot(1, 4, 4)
        ax4.set_title("inner ROI (color)")
        if show_roi_color and "inner_roi_bgr" in det.debug:
            ax4.imshow(cv2.cvtColor(det.debug["inner_roi_bgr"], cv2.COLOR_BGR2RGB))
        else:
            ax4.imshow(np.zeros((10, 10, 3), dtype=np.uint8))
        ax4.axis("off")
        plt.show()
    else:
        plt.figure(figsize=(7, 7))
        plt.imshow(cv2.cvtColor(annot, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

        if show_roi_color and det.debug is not None and "inner_roi_bgr" in det.debug:
            plt.figure(figsize=(6, 6))
            plt.title("inner ROI (color)")
            plt.imshow(cv2.cvtColor(det.debug["inner_roi_bgr"], cv2.COLOR_BGR2RGB))
            plt.axis("off")
            plt.show()

    # --- Show faces & pips per die (optional) ---
    if show_faces and full:
        for i, r in enumerate(full, start=1):
            face = r["face"]
            face_mask = r["face_mask"]
            pips_mask = r["pips_mask"]
            plt.figure(figsize=(12, 3))
            plt.subplot(1, 3, 1)
            plt.title(f"die {i} face (pips={r['pip_count']}, raw={r['raw_count']})")
            plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.title("face_mask")
            plt.imshow(face_mask, cmap="gray")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.title("pips_mask")
            plt.imshow(pips_mask, cmap="gray")
            plt.axis("off")
            plt.show()

# UI layout
ui_left = widgets.VBox([
    frame_idx_w,
    show_masks_w, show_faces_w, show_roi_color_w, show_scored_w,
    widgets.HTML("<b>Dice region (inner ROI)</b>"),
    ab_thr_w, L_min_w, L_q_w, open_k_w, close_k_w, close_it_w,
    min_area_w, max_area_w, max_aspect_w, pad_frac_w,
    widgets.HTML("<b>Square checks</b>"),
    use_rot_w, max_rot_aspect_w, min_extent_w, require_quad_w,
    widgets.HTML("<b>Selection</b>"),
    keep_top_k_w, max_score_cand_w, score_with_pips_w,
])

ui_mid = widgets.VBox([
    widgets.HTML("<b>Top-face crop</b>"),
    tf_ab_thr_w, tf_L_min_w, tf_L_q_w, tf_open_w, tf_close_w, tf_close_it_w,
    tf_pad_w, tf_out_w,
])

ui_right = widgets.VBox([
    widgets.HTML("<b>Pip counting</b>"),
    pip_ab_thr_w, pip_L_lo_w, pip_open_w, pip_close_w, pip_close_it_w,
    border_w, min_pip_a_w, max_pip_a_w,
])

controls = widgets.HBox([ui_left, ui_mid, ui_right])
display(controls)

out = widgets.interactive_output(
    run_debug,
    dict(
        frame=frame_idx_w,
        region_ab_thr=ab_thr_w,
        region_L_min=L_min_w,
        region_L_q=L_q_w,
        region_open_k=open_k_w,
        region_close_k=close_k_w,
        region_close_it=close_it_w,
        min_area_frac=min_area_w,
        max_area_frac=max_area_w,
        max_aspect=max_aspect_w,
        pad_frac=pad_frac_w,

        use_rotated_rect=use_rot_w,
        max_rot_aspect=max_rot_aspect_w,
        min_extent=min_extent_w,
        require_quad_like=require_quad_w,

        keep_top_k=keep_top_k_w,
        max_to_score=max_score_cand_w,
        score_with_pips=score_with_pips_w,

        face_ab_thr=tf_ab_thr_w,
        face_L_min=tf_L_min_w,
        face_L_q=tf_L_q_w,
        face_open_k=tf_open_w,
        face_close_k=tf_close_w,
        face_close_it=tf_close_it_w,
        face_pad=tf_pad_w,
        face_out_size=tf_out_w,

        pip_ab_thr=pip_ab_thr_w,
        pip_L_lo=pip_L_lo_w,
        pip_open_k=pip_open_w,
        pip_close_k=pip_close_w,
        pip_close_it=pip_close_it_w,
        border_frac=border_w,
        min_pip_area=min_pip_a_w,
        max_pip_area=max_pip_a_w,

        show_masks=show_masks_w,
        show_faces=show_faces_w,
        show_roi_color=show_roi_color_w,
        show_scored=show_scored_w,
    )
)
display(out)


Output()